In [1]:
from pyspark.sql import SparkSession,Row,DataFrame
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
import pyspark.sql.functions as F
import traceback

builder = SparkSession \
    .builder \
    .appName("Data with Nikk the Greek Spark Session") \
    .master("local[4]") \
    .config("spark.jars.packages", "uk.co.gresearch.spark:spark-extension_2.12:2.11.0-3.5") \
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.warehouse.dir", "/Users/eduardoalberto/LoadFile/repository/deltaTable/") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark


25/02/25 23:36:51 WARN Utils: Your hostname, MacBook-Pro-de-Eduardo.local resolves to a loopback address: 127.0.0.1; using 192.168.3.180 instead (on interface en0)
25/02/25 23:36:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/eduardoalberto/opt/spark-3.5.4/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/eduardoalberto/.ivy2/cache
The jars for the packages stored in: /Users/eduardoalberto/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8174334f-8559-41e2-9b8f-0d0f3924d61e;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 238ms :: artifacts dl 12ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default

PySpark Version :3.5.4
PySpark Version :3.5.4


In [ ]:
path_delta = "/Users/eduardoalberto/LoadFile/dataDelta/movie"
df = spark.read.parquet("/Users/eduardoalberto/LoadFile/parquet/movie")
df.write.format("delta").mode("overwrite").save(path_delta)

### Dataframe

In [ ]:
spark.read.format("delta").load(path_delta).show(truncate=False)

In [2]:
dfs = spark.read.option("delimiter",';')\
                .option("header", "True")\
                .option("inferSchema", "True")\
                .csv("/Users/eduardoalberto/LoadFile/part-00000-055103f0-b275-4e27-b667-0c2c25d0636a-c000.csv")
# dfs.printSchema()
# dfs.toPandas()

In [7]:
df01 = dfs.groupBy("name","imdb_id","overview","revenue","runtime","status","title","vote_average","vote_count","popularity","name_geners")\
          .agg(F.count("title").alias("total"))\
          .withColumn("dt_ref_carga", F.current_date())

df02 = dfs.groupBy("name","imdb_id","overview","revenue","runtime","status","title","vote_average","vote_count","popularity","name_geners")\
          .agg(F.count("title").alias("total"))\
          .withColumn("dt_ref_carga", F.current_date()+2)

df02.toPandas()

,name,imdb_id,overview,revenue,runtime,status,title,vote_average,vote_count,popularity,name_geners,total,dt_ref_carga
0,None,tt0119116,"In 2257, a taxi driver is unintentionally give...",263920180,126.0,Released,The Fifth Element,7,3962,24.305260,Drama,11966,2025-02-18
1,None,tt0078788,"At the height of the Vietnam war, Captain Benj...",89460381,153.0,Released,Apocalypse Now,8,2112,13.596300,Action,4489,2025-02-18
2,Before... Collection,tt0381681,Nine years ago two strangers met by chance and...,15992615,80.0,Released,Before Sunset,7,734,7.048957,Crime,1682,2025-02-18
3,Heart of Gold Collection,tt0168629,"Selma, a Czech immigrant on the verge of blind...",40031879,140.0,Released,Dancer in the Dark,7,392,10.684806,Animation,1124,2025-02-18
4,Deuce Bigalow Collection,tt0205000,"Deuce Bigalow is a less than attractive, down ...",65535067,88.0,Released,Deuce Bigalow: Male Gigolo,5,314,6.567794,Music,487,2025-02-18
5,None,tt0185125,A single mother in Madrid sees her only son di...,67872296,101.0,Released,All About My Mother,7,337,10.000915,Documentary,3415,2025-02-18
6,None,tt0169547,"Lester Burnham, a depressed suburban father in...",356296601,122.0,Released,American Beauty,7,3438,20.726578,Fantasy,704,2025-02-18
7,Finding Nemo Collection,tt0266543,"Nemo, an adventurous young clownfish, is unexp...",940335536,100.0,Released,Finding Nemo,7,6292,25.497794,Adventure,1514,2025-02-18
8,In China They Eat Dogs Collection,tt0246692,"The last wish of the dying ""Monk"" is for his f...",0,95.0,Released,Old Men in New Cars: In China They Eat Dogs II,6,37,4.003761,Family,524,2025-02-18
9,None,tt0268437,A lawyer becomes a fisherman from frustration....,0,97.0,Released,The Man Who Sued God,6,23,1.741405,Mystery,554,2025-02-18


In [4]:
arqDelta = "/Users/eduardoalberto/LoadFile/dataDelta/ratingMovie"
# df = spark.read.parquet("/Users/eduardoalberto/LoadFile/parquet/movie")
df01.write.format("delta").mode("overwrite").partitionBy("dt_ref_carga").option("overwriteSchema", "true").saveAsTable("tb_ratingMovie")
df02.write.format("delta").mode("overwrite").partitionBy("dt_ref_carga").option("overwriteSchema", "true").saveAsTable("st_ratingMovie")


In [6]:
spark.table("st_ratingMovie").filter("imdb_id = 'tt0036818'").toPandas()

# spark.read.format("delta").load(arqDelta).toPandas()

,name,imdb_id,overview,revenue,runtime,status,title,vote_average,vote_count,popularity,name_geners,total,dt_ref_carga
0,None,tt0036818,Die Feuerzangenbowle from Director Helmut Weis...,0,97.0,Released,The Punch Bowl,6,41,2.098595,Science Fiction,646,2025-02-19


In [19]:
spark.sql(""" update st_ratingMovie
              set imdb_id = "tt0036818",
                  revenue = 1,
                  vote_average = 7,
                  vote_count = 42,
                  dt_ref_carga = current_date()+1
               where imdb_id = 'tt0036818'    
""")

spark.table("st_ratingMovie").filter("imdb_id = 'tt0036818' ").toPandas()

,name,imdb_id,overview,revenue,runtime,status,title,vote_average,vote_count,popularity,name_geners,total,dt_ref_carga
0,None,tt0036818,Die Feuerzangenbowle from Director Helmut Weis...,1,97.0,Released,The Punch Bowl,7,42,2.098595,Science Fiction,646,2025-02-21


### MERGE UPDATE SPARK SQL

In [ ]:
spark.sql("""merge into tb_ratingMovie T1
             using st_ratingMovie T2
             on T1.imdb_id = T2.imdb_id
             when matched then update set *
             when not matched by target then insert *
""")

In [ ]:
# spark.table("tb_ratingMovie").select("dt_ref_carga").show()

spark.table("tb_ratingMovie").filter("imdb_id = 'tt0036818' ").toPandas()

### MERGE UPDATE PYSPARK

In [21]:
dftg = DeltaTable.forPath(spark, "/Users/eduardoalberto/LoadFile/repository/deltaTable/tb_ratingmovie")
dfsr = spark.read.format("delta").load("/Users/eduardoalberto/LoadFile/repository/deltaTable/st_ratingmovie")


(dftg.alias("dftg")
     .merge(dfsr.alias("dfsr"),"dftg.imdb_id = dfsr.imdb_id")
     .whenMatchedUpdateAll()
     .whenNotMatchedInsertAll()
     .whenNotMatchedBySourceDelete()
     .execute()
 )



In [3]:
spark.table("tb_ratingmovie").filter("imdb_id = 'tt0036818'").toPandas()

,name,imdb_id,overview,revenue,runtime,status,title,vote_average,vote_count,popularity,name_geners,total,dt_ref_carga
0,None,tt0036818,Die Feuerzangenbowle from Director Helmut Weis...,0,97.0,Released,The Punch Bowl,6,41,2.098595,Science Fiction,646,2025-02-23


In [ ]:



import os.path as path


def LaodDelta (dir):

    try:
        path.exists(dir)
        df = spark.read.option("delimiter",';').option("header", "True").option("inferSchema", "True").csv(dir)
        df01 = df.groupBy("name","imdb_id","overview","revenue","runtime","status","title","vote_average","vote_count","popularity","name_geners")\
                 .agg(F.count("title").alias("total"))\
                 .withColumn("dt_ref_carga", F.current_date())

        df02 = df.groupBy("name","imdb_id","overview","revenue","runtime","status","title","vote_average","vote_count","popularity","name_geners")\
                 .agg(F.count("title").alias("total"))\
                 .withColumn("dt_ref_carga", F.current_date()+2)
    
        df01.write.format("delta").mode("overwrite").partitionBy("dt_ref_carga").option("overwriteSchema", "true").saveAsTable("tb_ratingMovie")
        df02.write.format("delta").mode("overwrite").partitionBy("dt_ref_carga").option("overwriteSchema", "true").saveAsTable("st_ratingMovie")
 
    except Exception as e:
        print(f"Ocorreu o seguinte erro: {e}")
        traceback.print_exc()
    return None

def MergeDelta(dftg,dfsr):
    try:
        dftg.alias("dftg")\
            .merge(dfsr.alias("dfsr"),"dftg.imdb_id = dfsr.imdb_id")\
            .whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
            .whenNotMatchedBySourceDelete()\
            .execute()
        
    except Exception as e:
        print(f"Ocorreu o seguinte erro: {e}")
        traceback.print_exc()
    return None
    


#################################################################################################################################################################

dftg = DeltaTable.forPath(spark, "/Users/eduardoalberto/LoadFile/repository/deltaTable/tb_ratingmovie")
dfsr = spark.read.format("delta").load("/Users/eduardoalberto/LoadFile/repository/deltaTable/st_ratingmovie")


dir = "/Users/eduardoalberto/LoadFile/part-00000-055103f0-b275-4e27-b667-0c2c25d0636a-c000.csv"

LaodDelta(dir)
MergeDelta(dftg,dfsr)
